# Benchmark 1

We study the Hamiltonian of the Heisenberg model with periodic boundary conditions.

In [1]:
using CondensedMatterSOS
@spin σ[1:3]
heisenberg_hamiltonian(σ, true)

# Let's pick a solver from [this list](https://jump.dev/JuMP.jl/dev/installation/#Getting-Solvers).

using CSDP
solver = optimizer_with_attributes(
    () -> MOIU.CachingOptimizer(MOIU.UniversalFallback(MOIU.Model{Float64}()), CSDP.Optimizer()),
    MOI.Silent() => false,
)

MathOptInterface.OptimizerWithAttributes(Main.##258.var"#1#2"(), Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.Silent() => false])

We can compute a lower bound `-2√2` to the ground state energy as follow:

In [2]:
include(joinpath(dirname(dirname(pathof(CondensedMatterSOS))), "examples", "symmetry.jl"))
function hamiltonian_energy(N, maxdegree, solver; symmetry=true, consecutive=false, kws...)
    @spin σ[1:N]
    function action(mono::CondensedMatterSOS.SpinMonomial, el::DirectSum)
        isempty(mono.variables) && return 1 * mono
        sign = 1
        vars = map(values(mono.variables)) do var
            rel_id = var.id - σ[1][1].id
            rel_index = var.index + 1
            @assert σ[rel_index][rel_id + 1] == var
            id = ((rel_id + el.c.id) % el.c.n) + σ[1][1].id
            index = (rel_index^el.k.p) - 1
            new_var = CondensedMatterSOS.SpinVariable(id, index)
            if el.k.k.id != 0 && el.k.k.id != index + 1
                sign *= -1
            end
            return new_var
        end
        return sign * CondensedMatterSOS.SpinMonomial(vars)
    end
    function action(term::CondensedMatterSOS.SpinTerm, el::DirectSum)
        return MP.coefficient(term) * action(MP.monomial(term), el)
    end
    function action(poly::CondensedMatterSOS.SpinPolynomial, el::DirectSum)
        return MP.polynomial([action(term, el) for term in MP.terms(poly)])
    end
    H = heisenberg_hamiltonian(σ, true)
    G = Lattice1Group(N)
    cone = NonnegPolyInnerCone{SumOfSquares.COI.HermitianPositiveSemidefiniteConeTriangle}()
    @assert iseven(maxdegree)
    cert = SumOfSquares.Certificate.FixedBasis(
        cone,
        MonomialBasis(MP.monomials(σ[1], 0:div(maxdegree, 2), consecutive=consecutive)),
    )
    display(cert.basis.monomials)
    certificate = SymmetricIdeal(
        cert,
        G,
        action,
    )
    if symmetry
        energy(H, maxdegree, solver; certificate = certificate, kws...)
    else
        energy(H, maxdegree, solver; kws...)
    end
end
bound, gram, ν = hamiltonian_energy(
    2,
    2,
    solver,
    symmetry = false,
    sparsity = NoSparsity(),
)
bound

Iter: 14 Ap: 9.58e-01 Pobj: -1.2000000e+01 Ad: 9.59e-01 Dobj: -1.2000000e+01 
Success: SDP solved
Primal objective value: -1.2000000e+01 
Dual objective value: -1.2000000e+01 
Relative primal infeasibility: 5.45e-13 
Relative dual infeasibility: 2.96e-09 
Real Relative Gap: 3.15e-09 
XZ Relative Gap: 5.02e-09 
DIMACS error measures: 1.43e-12 0.00e+00 1.76e-08 0.00e+00 3.15e-09 5.02e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -5.6411147e+01 Ad: 4.06e-01 Dobj: -4.2832684e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -5.4760276e+01 Ad: 9.64e-01 Dobj: -7.6705709e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -2.1723633e+01 Ad: 8.78e-01 Dobj: -1.3767694e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -8.4915625e+00 Ad: 7.01e-01 Dobj: -3.2534741e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -6.6196805e+00 Ad: 8.87e-01 Dobj: -5.5119401e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -6.0450487e+00 Ad: 9.46e-01 Dobj: -5.9459371e+00 
Iter:  7 Ap: 1.00e+00 Pobj: -6.0023373e+00 Ad: 9.58e-0

7-element Vector{CondensedMatterSOS.SpinMonomial}:
 1
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂

-6.000000022862929

We can see that the moment matrix uses all monomials:

In [3]:
ν.basis.monomials

7-element Vector{CondensedMatterSOS.SpinMonomial}:
 1
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂

# Symmetry reduction

We can reduce the computation using symmetry reduction as follows.

In [4]:
using CondensedMatterSOS

bound, gram, ν = hamiltonian_energy(
    2,
    2,
    solver,
)
bound

Iter: 11 Ap: 9.59e-01 Pobj: -6.0000000e+00 Ad: 9.59e-01 Dobj: -6.0000000e+00 
Success: SDP solved
Primal objective value: -6.0000000e+00 
Dual objective value: -6.0000000e+00 
Relative primal infeasibility: 1.15e-15 
Relative dual infeasibility: 2.03e-09 
Real Relative Gap: 1.97e-09 
XZ Relative Gap: 4.05e-09 
DIMACS error measures: 1.72e-15 0.00e+00 3.15e-09 0.00e+00 1.97e-09 4.05e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.43e-01 Pobj: -9.4276778e+00 Ad: 7.72e-01 Dobj: -1.5875894e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -1.8664477e+01 Ad: 9.31e-01 Dobj: -3.2456338e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -8.9497605e+00 Ad: 8.97e-01 Dobj: -4.8223362e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -6.2804877e+00 Ad: 9.09e-01 Dobj: -5.7836416e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -6.0142901e+00 Ad: 9.40e-01 Dobj: -5.9808390e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -6.0006773e+00 Ad: 9.61e-01 Dobj: -5.9989293e+00 
Iter:  7 Ap: 1.00e+00 Pobj: -6.0000443e+00 Ad: 9.88e-0

7-element Vector{CondensedMatterSOS.SpinMonomial}:
 1
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂

-6.000000005297709

The reduction is obtained by block diagonalizing with a change of polynomial
basis to the isotypical basis.

In [5]:
display([M.basis.polynomials for M in ν.sub_moment_matrices])

[M.basis.polynomials for M in ν.sub_moment_matrices]

7-element Vector{Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}}:
 [σˣ₁ + σˣ₂]
 [σʸ₁ + σʸ₂]
 [σᶻ₁ + σᶻ₂]
 [σˣ₁ + -σˣ₂]
 [σʸ₁ + -σʸ₂]
 [σᶻ₁ + -σᶻ₂]
 [(1.0 + 0.0im)]

7-element Vector{Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}}:
 [σˣ₁ + σˣ₂]
 [σʸ₁ + σʸ₂]
 [σᶻ₁ + σᶻ₂]
 [σˣ₁ + -σˣ₂]
 [σʸ₁ + -σʸ₂]
 [σᶻ₁ + -σᶻ₂]
 [(1.0 + 0.0im)]

Let's try this for 3 sites. First without symmetry.

In [6]:
bound, gram, ν = hamiltonian_energy(
    3,
    2,
    solver,
    symmetry = false,
)
@show bound

Iter: 10 Ap: 9.69e-01 Pobj: -6.0000000e+00 Ad: 9.69e-01 Dobj: -6.0000000e+00 
Success: SDP solved
Primal objective value: -6.0000000e+00 
Dual objective value: -6.0000000e+00 
Relative primal infeasibility: 9.73e-16 
Relative dual infeasibility: 5.85e-10 
Real Relative Gap: 5.00e-10 
XZ Relative Gap: 9.73e-10 
DIMACS error measures: 1.45e-15 0.00e+00 1.41e-09 0.00e+00 5.00e-10 9.73e-10
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -5.6096216e+01 Ad: 4.07e-01 Dobj: -4.5047371e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -5.5003096e+01 Ad: 9.56e-01 Dobj: -7.7051477e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -2.4832117e+01 Ad: 8.97e-01 Dobj: -1.2354912e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -6.5354955e+00 Ad: 8.45e-01 Dobj: -2.7588323e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -4.8443775e+00 Ad: 9.03e-01 Dobj: -4.2536145e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -4.5235955e+00 Ad: 9.40e-01 Dobj: -4.4765103e+00 
Iter:  7 Ap: 1.00e+00 Pobj: -4.5013948e+00 Ad: 9.60e-0

10-element Vector{CondensedMatterSOS.SpinMonomial}:
 1
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂
 σˣ₃
 σʸ₃
 σᶻ₃

-4.500000019876194

Now with symmetry.

In [7]:
bound, gram, ν = hamiltonian_energy(
    3,
    2,
    solver,
)
@show bound

Iter: 11 Ap: 9.58e-01 Pobj: -4.5000000e+00 Ad: 9.58e-01 Dobj: -4.5000000e+00 
Success: SDP solved
Primal objective value: -4.5000000e+00 
Dual objective value: -4.5000000e+00 
Relative primal infeasibility: 4.61e-16 
Relative dual infeasibility: 1.77e-09 
Real Relative Gap: 2.08e-09 
XZ Relative Gap: 3.52e-09 
DIMACS error measures: 9.22e-16 0.00e+00 4.63e-09 0.00e+00 2.08e-09 3.52e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.43e-01 Pobj: -5.2740518e+00 Ad: 8.34e-01 Dobj: -5.6409413e-01 
Iter:  2 Ap: 1.00e+00 Pobj: -2.3913022e+01 Ad: 9.25e-01 Dobj: -2.6711480e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -7.3750123e+00 Ad: 9.22e-01 Dobj: -3.5379465e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -4.8052420e+00 Ad: 8.88e-01 Dobj: -4.3246911e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -4.5185578e+00 Ad: 9.29e-01 Dobj: -4.4820031e+00 
Iter:  6 Ap: 1.00e+00 Pobj: -4.5011355e+00 Ad: 9.57e-01 Dobj: -4.4988629e+00 
Iter:  7 Ap: 1.00e+00 Pobj: -4.5000803e+00 Ad: 9.80e-0

10-element Vector{CondensedMatterSOS.SpinMonomial}:
 1
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂
 σˣ₃
 σʸ₃
 σᶻ₃

-4.500000010264298

Let's look at the isotypical basis.

In [8]:
display([M.basis.polynomials for M in ν.sub_moment_matrices])

10-element Vector{Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}}:
 [σˣ₁ + σˣ₂ + σˣ₃]
 [σʸ₁ + σʸ₂ + σʸ₃]
 [σᶻ₁ + σᶻ₂ + σᶻ₃]
 [σˣ₁ + (-0.4999999999999998 + 0.8660254037844387im)σˣ₂ + (-0.5000000000000004 - 0.8660254037844385im)σˣ₃]
 [σʸ₁ + (-0.4999999999999998 + 0.8660254037844387im)σʸ₂ + (-0.5000000000000004 - 0.8660254037844385im)σʸ₃]
 [σᶻ₁ + (-0.4999999999999998 + 0.8660254037844387im)σᶻ₂ + (-0.5000000000000004 - 0.8660254037844385im)σᶻ₃]
 [σˣ₁ + (-0.5000000000000004 - 0.8660254037844385im)σˣ₂ + (-0.4999999999999998 + 0.8660254037844387im)σˣ₃]
 [σʸ₁ + (-0.5000000000000004 - 0.8660254037844385im)σʸ₂ + (-0.4999999999999998 + 0.8660254037844387im)σʸ₃]
 [σᶻ₁ + (-0.5000000000000004 - 0.8660254037844385im)σᶻ₂ + (-0.4999999999999998 + 0.8660254037844387im)σᶻ₃]
 [(1.0 + 0.0im)]

Now let's define a function for our common use case.

In [9]:
function f(L, d=1, consecutive=false; symmetry=true)
    @show L
    println("***")
    @show d
    bound, gram, ν = @time hamiltonian_energy(
        L,
        2d,
        solver,
        consecutive=consecutive,
        symmetry=symmetry,
    )
    @show bound
    for M in ν.sub_moment_matrices
        display(round.(M.basis.polynomials, digits=6))
    end
    println("E/N = ", bound / L)
    println("------------------------------------")
end

f(6, 1, true)

L = 6
***
d = 1
Iter: 10 Ap: 9.68e-01 Pobj: -4.5000000e+00 Ad: 9.68e-01 Dobj: -4.5000000e+00 
Success: SDP solved
Primal objective value: -4.5000000e+00 
Dual objective value: -4.5000000e+00 
Relative primal infeasibility: 5.84e-16 
Relative dual infeasibility: 1.24e-09 
Real Relative Gap: 1.22e-09 
XZ Relative Gap: 1.96e-09 
DIMACS error measures: 1.17e-15 0.00e+00 3.35e-09 0.00e+00 1.22e-09 1.96e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.41e-01 Pobj: -4.0668056e+00 Ad: 8.61e-01 Dobj: -5.2018015e-01 
Iter:  2 Ap: 1.00e+00 Pobj: -5.6056177e+01 Ad: 9.36e-01 Dobj: -5.5398751e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -2.2189805e+01 Ad: 8.38e-01 Dobj: -8.5447620e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -2.1429608e+01 Ad: 7.30e-01 Dobj: -1.4881165e+01 
Iter:  5 Ap: 1.00e+00 Pobj: -1.8342881e+01 Ad: 8.69e-01 Dobj: -1.7133133e+01 
Iter:  6 Ap: 1.00e+00 Pobj: -1.8022311e+01 Ad: 9.28e-01 Dobj: -1.7891496e+01 
Iter:  7 Ap: 1.00e+00 Pobj: -1.8001586

19-element Vector{CondensedMatterSOS.SpinMonomial}:
 1
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂
 σˣ₃
 σʸ₃
 σᶻ₃
 σˣ₄
 σʸ₄
 σᶻ₄
 σˣ₅
 σʸ₅
 σᶻ₅
 σˣ₆
 σʸ₆
 σᶻ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σˣ₁ + σˣ₂ + σˣ₃ + σˣ₄ + σˣ₅ + σˣ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σʸ₁ + σʸ₂ + σʸ₃ + σʸ₄ + σʸ₅ + σʸ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σᶻ₁ + σᶻ₂ + σᶻ₃ + σᶻ₄ + σᶻ₅ + σᶻ₆

...

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σᶻ₁ + (0.5 - 0.866025im)σᶻ₂ + (-0.5 - 0.866025im)σᶻ₃ + -σᶻ₄ + (-0.5 + 0.866025im)σᶻ₅ + (0.5 + 0.866025im)σᶻ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σˣ₁ + -σˣ₂ + σˣ₃ + -σˣ₄ + σˣ₅ + -σˣ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σʸ₁ + -σʸ₂ + σʸ₃ + -σʸ₄ + σʸ₅ + -σʸ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σᶻ₁ + -σᶻ₂ + σᶻ₃ + -σᶻ₄ + σᶻ₅ + -σᶻ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 (1.0 + 0.0im)

Now with `d = 2`.

In [10]:
f(6, 2, true)

L = 6
***
d = 2
Iter: 11 Ap: 9.58e-01 Pobj: -1.8000000e+01 Ad: 9.58e-01 Dobj: -1.8000000e+01 
Success: SDP solved
Primal objective value: -1.8000000e+01 
Dual objective value: -1.8000000e+01 
Relative primal infeasibility: 1.92e-14 
Relative dual infeasibility: 1.59e-09 
Real Relative Gap: 2.41e-09 
XZ Relative Gap: 3.09e-09 
DIMACS error measures: 5.04e-14 0.00e+00 5.43e-09 0.00e+00 2.41e-09 3.09e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 8.87e-01 Pobj: -5.4717903e+00 Ad: 8.18e-01 Dobj:  1.3890632e+00 
Iter:  2 Ap: 9.60e-01 Pobj: -3.4653075e+02 Ad: 3.99e-01 Dobj: -9.6813687e-01 
Iter:  3 Ap: 9.19e-01 Pobj: -2.2340004e+02 Ad: 9.13e-01 Dobj: -2.5587359e+00 
Iter:  4 Ap: 9.54e-01 Pobj: -3.8762155e+01 Ad: 8.96e-01 Dobj: -3.2505826e+00 
Iter:  5 Ap: 9.53e-01 Pobj: -2.3654466e+01 Ad: 7.29e-01 Dobj: -7.7112448e+00 
Iter:  6 Ap: 9.43e-01 Pobj: -1.5590694e+01 Ad: 7.88e-01 Dobj: -9.8874655e+00 
Iter:  7 Ap: 1.00e+00 Pobj: -1.3151596

73-element Vector{CondensedMatterSOS.SpinMonomial}:
 1
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂
 σˣ₃
 σʸ₃
 σᶻ₃
 ⋮
 σˣ₁σˣ₆
 σʸ₁σˣ₆
 σᶻ₁σˣ₆
 σˣ₁σʸ₆
 σʸ₁σʸ₆
 σᶻ₁σʸ₆
 σˣ₁σᶻ₆
 σʸ₁σᶻ₆
 σᶻ₁σᶻ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σˣ₁σˣ₂ + σˣ₁σˣ₆ + -σᶻ₁σᶻ₂ + -σᶻ₁σᶻ₆ + σˣ₂σˣ₃ + -σᶻ₂σᶻ₃ + σˣ₃σˣ₄ + -σᶻ₃σᶻ₄ + σˣ₄σˣ₅ + -σᶻ₄σᶻ₅ + σˣ₅σˣ₆ + -σᶻ₅σᶻ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σʸ₁σʸ₂ + σʸ₁σʸ₆ + -σᶻ₁σᶻ₂ + -σᶻ₁σᶻ₆ + σʸ₂σʸ₃ + -σᶻ₂σᶻ₃ + σʸ₃σʸ₄ + -σᶻ₃σᶻ₄ + σʸ₄σʸ₅ + -σᶻ₄σᶻ₅ + σʸ₅σʸ₆ + -σᶻ₅σᶻ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σˣ₁σˣ₂ + (-0.5 - 0.866025im)σˣ₁σˣ₆ + -σᶻ₁σᶻ₂ + (0.5 + 0.866025im)σᶻ₁σᶻ₆ + (-0.5 + 0.866025im)σˣ₂σˣ₃ + (0.5 - 0.866025im)σᶻ₂σᶻ₃ + (-0.5 - 0.866025im)σˣ₃σˣ₄ + (0.5 + 0.866025im)σᶻ₃σᶻ₄ + σˣ₄σˣ₅ + -σᶻ₄σᶻ₅ + (-0.5 + 0.866025im)σˣ₅σˣ₆ + (0.5 - 0.866025im)σᶻ₅σᶻ₆

...

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σˣ₁σᶻ₂ + (-0.5 - 0.866025im)σˣ₁σᶻ₆ + -σᶻ₁σˣ₂ + (0.5 + 0.866025im)σᶻ₁σˣ₆ + (0.5 - 0.866025im)σˣ₂σᶻ₃ + (-0.5 + 0.866025im)σᶻ₂σˣ₃ + (-0.5 - 0.866025im)σˣ₃σᶻ₄ + (0.5 + 0.866025im)σᶻ₃σˣ₄ + -σˣ₄σᶻ₅ + σᶻ₄σˣ₅ + (-0.5 + 0.866025im)σˣ₅σᶻ₆ + (0.5 - 0.866025im)σᶻ₅σˣ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σʸ₁σᶻ₂ + (-0.5 - 0.866025im)σʸ₁σᶻ₆ + -σᶻ₁σʸ₂ + (0.5 + 0.866025im)σᶻ₁σʸ₆ + (0.5 - 0.866025im)σʸ₂σᶻ₃ + (-0.5 + 0.866025im)σᶻ₂σʸ₃ + (-0.5 - 0.866025im)σʸ₃σᶻ₄ + (0.5 + 0.866025im)σᶻ₃σʸ₄ + -σʸ₄σᶻ₅ + σᶻ₄σʸ₅ + (-0.5 + 0.866025im)σʸ₅σᶻ₆ + (0.5 - 0.866025im)σᶻ₅σʸ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σˣ₁σʸ₂ + σˣ₁σʸ₆ + -σʸ₁σˣ₂ + -σʸ₁σˣ₆ + -σˣ₂σʸ₃ + σʸ₂σˣ₃ + σˣ₃σʸ₄ + -σʸ₃σˣ₄ + -σˣ₄σʸ₅ + σʸ₄σˣ₅ + σˣ₅σʸ₆ + -σʸ₅σˣ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σˣ₁σᶻ₂ + σˣ₁σᶻ₆ + -σᶻ₁σˣ₂ + -σᶻ₁σˣ₆ + -σˣ₂σᶻ₃ + σᶻ₂σˣ₃ + σˣ₃σᶻ₄ + -σᶻ₃σˣ₄ + -σˣ₄σᶻ₅ + σᶻ₄σˣ₅ + σˣ₅σᶻ₆ + -σᶻ₅σˣ₆

1-element Vector{CondensedMatterSOS.SpinPolynomial{ComplexF64}}:
 σʸ₁σᶻ₂ + σʸ₁σᶻ₆ + -σᶻ₁σʸ₂ + -σᶻ₁σʸ₆ + -σʸ₂σᶻ₃ + σᶻ₂σʸ₃ + σʸ₃σᶻ₄ + -σᶻ₃σʸ₄ + -σʸ₄σᶻ₅ + σᶻ₄σʸ₅ + σʸ₅σᶻ₆ + -σᶻ₅σʸ₆

| id     | irep 1 | irep 2 | irep 3 | irep 4 |
|--------|--------|--------|--------|--------|
| degree | 2      | 3      | 1      | 3      |
| mult 2 | 1      | 3      | 2      | 1      |
| mult 3 | 3      | 6      | 4      | 3      |
| mult 4 | 6      | 10     | 7      | 6      |
| mult 5 | 10     | 15     | 11     | 10     |
| mult 6 | 15     | 21     | 16     | 15     |
| mult 7 | 21     | 28     | 22     | 21     |

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*